<a href="https://colab.research.google.com/github/gtoubian/cce/blob/main/1_7_SQL_Part_3_%2B_Review_Lecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In the last lecture, we went over a couple of the basic clauses in SQL. In today's lecture we will be doing more of the same.

In [1]:
import sqlite3 as sql
conn = sql.connect('lecture.db')
import pandas as pd
c = conn.cursor()

In [2]:
df = pd.DataFrame(([['Alex', 18, 'Student', 5.5], ['Isabella', 24, 'Accountant', 5], ['George', 33, 'Elementary School Teacher', 3], ['Stacey',28,'Elementary School Teacher',4], ['Ayesha', 21, 'Student', 2.8]]),
                   columns=['Name', 'Age', 'Occupation', 'Distance Ran'])

df.to_sql('Runners', con = conn)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [4]:
df.head()

,Name,Age,Occupation,Distance Ran
0,Alex,18,Student,5.5
1,Isabella,24,Accountant,5.0
2,George,33,Elementary School Teacher,3.0
3,Stacey,28,Elementary School Teacher,4.0
4,Ayesha,21,Student,2.8


Above, we have created a dataframe with the Name, Age, Occupation and number of KMs that were run by participants in a half hour long jog that we loaded into an SQL Schema.

We can use the following functions to calculate different results for us such as the Mean, Max, Min, Sum and count of entries in the Table.

In [5]:
pd.read_sql('SELECT AVG(`Distance Ran`), MAX(Age), MIN(`Distance Ran`), SUM(AGE), count(*) from Runners', con=conn)

,AVG(`Distance Ran`),MAX(Age),MIN(`Distance Ran`),SUM(AGE),count(*)
0,4.06,33,2.8,124,5


In [7]:
pd.read_sql("SELECT Occupation, count(*) as Number, AVG(`Distance Ran`) as AVG_DIST, MAX(Age) as MAX_AGE from Runners group by Occupation", con=conn)

,Occupation,Number,AVG_DIST,MAX_AGE
0,Accountant,1,5.00,24
1,Elementary School Teacher,2,3.50,33
2,Student,2,4.15,21


The same functions can compute different results if a 'GROUP BY' Clause is used.

,Occupation,Number,AVG_DIST,MAX_AGE
0,Accountant,1,5.00,24
1,Elementary School Teacher,2,3.50,33
2,Student,2,4.15,21


For the next exercises, I have loaded a dataset on free and paid parking locations in Montreal taken from the city of Montreal website.

In [11]:
url='https://data.montreal.ca/dataset/575ecf37-9097-44cd-817f-a2fbd8de314b/resource/16288929-b544-47c8-9e2f-93b162fc04c6/download/stationnements_h_2020_2021.csv'
df = pd.read_csv(url)
df.head(2)

,OBJECTID,ARRONDISSEMENT,NBR_PLA,X,Y,ID_STA,JURIDICTION,EMPLACEMENT,LOCATION,HEURES,HOURS,BOROUGH,NBR_PLA_I,NOTE_FR,NOTE_EN,TYPE_PAY,Latitude,Longitude
0,58,Ahunstic-Cartierville,17,292433.00,5046689.31,58,Stationnement de Montréal,Rue Fleury près de la rue Chambord (#335),Rue Fleury near Chambord St. (parking #335),15:00 à 07:00,3:00 pm to 07:00 am,Ahunstic-Cartierville,17,Section horaire,Hourly spaces,0.0,45.5600,-73.6584
1,91,Anjou,20,298302.08,5051493.88,91,Municipale,Parc Lucie-Bruneau - 7051 av. de l'Alsace,Lucie-Bruneau Park - 7051 de l'Alsace Ave,06:00 à 18:00,6:00 am to 6:00 pm,Anjou,20,NaN,NaN,0.0,45.6033,-73.5833


In [13]:
df.to_sql("Parking",con=conn)

Here, let's get a count of the confirmed free parking spots.

In [14]:
pd.read_sql("Select type_pay as Free_Or_Not, count(*) as Parking_Spots from Parking group by type_pay", con=conn)

,Free_Or_Not,Parking_Spots
0,NaN,2
1,0.0,135


In [15]:
pd.read_sql("Select Location from Parking where type_pay is Null", con = conn)

,LOCATION
0,Saint-Jean-Baptiste Blvd. and Victoria St corner
1,6th Ave. between Daudurand and Masson St. (#73)


#Between
The Between Clause is used in the where statement to specify rows that have a column value in between some lower and upper number.

In [17]:
pd.read_sql("Select Location, Juridiction, nbr_pla from Parking where nbr_pla between 10 and 20 limit 5", con = conn)

,LOCATION,JURIDICTION,NBR_PLA
0,Rue Fleury near Chambord St. (parking #335),Stationnement de Montréal,17
1,Lucie-Bruneau Park - 7051 de l'Alsace Ave,Municipale,20
2,André-Laurendeau Park - 8361 André-Laurendeau ...,Municipale,20
3,City Hall - 7701 Louis-H.Lafontaine Blvd,Municipale,20
4,Chénier Arena - 8200 Chénier Ave,Municipale,20


#Like
The Like Clause is also used after the where clause. The Like Clause helps us to pick out rows that contain a column value usually denoted with %a%. 

In [19]:
pd.read_sql("Select Location, Hours from Parking where Juridiction like 'Muni%' Limit 5", con = conn)

,LOCATION,HOURS
0,Lucie-Bruneau Park - 7051 de l'Alsace Ave,6:00 am to 6:00 pm
1,André-Laurendeau Park - 8361 André-Laurendeau ...,6:00 am to 6:00 pm
2,City Hall - 7701 Louis-H.Lafontaine Blvd,6:00 am to 6:00 pm
3,Chénier Arena - 8200 Chénier Ave,6:00 am to 6:00 pm
4,Chaumont Arena - 8750 Chaumont Ave,6:00 am to 6:00 pm


The next clause in SQL we will look at is the CASE clause. The Case Clause is similar to the if/else statements that we have seen in python where if a certain condition is met then SQL will execute a code. The format is as follows:

```
SELECT LOCATION,
       HOURS
       CASE TYPE_PAY 
           WHEN '0.0' 
               THEN 'Free' 
           ELSE 'Pay' 
       END Paid
FROM 
    Parking
```

#Subquery

So far we have been querying from a raw table of values and outputting them in new ones. But what if we wanted to pull values yet again from these new tables? Here we can use subqueries.

A subquery is a SELECT statement nested in another statement. See the following statement.



```
SELECT column_1
FROM table_1
WHERE column_1 = (
   SELECT column_1 
   FROM table_2
);
```
where the outer query is: 


```
SELECT column_1
  FROM table_1
 WHERE colum_1 =
```

and the inner column is:



```
(SELECT column_1
  FROM table_2)
```



#Department Highest Salary

The Employee table holds all employees. Every employee has an Id, a salary, and there is also a column for the department Id.

```
+----+-------+--------+--------------+
| Id | Name  | Salary | DepartmentId |
+----+-------+--------+--------------+
| 1  | Joe   | 70000  | 1            |
| 2  | Jim   | 90000  | 1            |
| 3  | Henry | 80000  | 2            |
| 4  | Sam   | 60000  | 2            |
| 5  | Max   | 90000  | 1            |
+----+-------+--------+--------------+
```

The Department table holds all departments of the company.

```
+----+----------+
| Id | Name     |
+----+----------+
| 1  | IT       |
| 2  | Sales    |
+----+----------+
```

Write a SQL query to find employees who have the highest salary in each of the departments. For the above tables, your SQL query should return the following rows (order of rows does not matter).

```
+------------+----------+--------+
| Department | Employee | Salary |
+------------+----------+--------+
| IT         | Max      | 90000  |
| IT         | Jim      | 90000  |
| Sales      | Henry    | 80000  |
+------------+----------+--------+
```

Max and Jim both have the highest salary in the IT department and Henry has the highest salary in the Sales department.

In [20]:
df1 = pd.DataFrame(([[1, 'Joe', 70000, 1], [2, 'Jim', 90000, 1], [3, 'Henry', 80000, 2], [4,'Sam',60000,2], [5, 'Max', 90000, 1]]),
                   columns=['Id', 'Name', 'Salary', 'DepartmentId'])
df2 = pd.DataFrame(([[1,'IT'],[2,'Sales']]), columns=['Id', 'Name'])

df1.to_sql('Employee', con = conn)
df2.to_sql('Department', con = conn)

#Review

So far in this module we have covered Data types/containers, Python Logic Flow, Functions, Math, Statistics, Numpy, Pandas, Classes and SQL. As we go further along we will be drawing on the lectures from module 1 as well as introducing you to new data analysis techniques. Please make sure to go over the lecture/workshop notebooks, rewatch lecture recordings (if needed) and practice technical interview questions on Leetcode/HackerRank to have a solid grasp of these basics. 



#Programming Questions

##How Many Numbers Are Smaller Than the Current Number

Given the array nums, for each nums[i] find out how many numbers in the array are smaller than it. That is, for each nums[i] you have to count the number of valid j's such that j != i and nums[j] < nums[i].

Return the answer in an array.

 

Example 1:


```
Input: nums = [8,1,2,2,3]
Output: [4,0,1,1,3]
Explanation: 
For nums[0]=8 there exist four smaller numbers than it (1, 2, 2 and 3). 
For nums[1]=1 does not exist any smaller number than it.
For nums[2]=2 there exist one smaller number than it (1). 
For nums[3]=2 there exist one smaller number than it (1). 
For nums[4]=3 there exist three smaller numbers than it (1, 2 and 2).
```



In [28]:
nums = [8,1,2,2,3]
def smaller(nums):
  for i in range(len(nums)):
    count = 0
    for j in range(len(nums)):
      if j!=i and nums[j]<nums[i]:
        count +=1
    output.append(count)
  return output

smaller(nums)

[4, 0, 1, 1, 3]

# Maximum Number of Coins You Can Get

There are 3n piles of coins of varying size, you and your friends will take piles of coins as follows:

In each step, you will choose any 3 piles of coins (not necessarily consecutive).
Of your choice, Alice will pick the pile with the maximum number of coins.
You will pick the next pile with maximum number of coins.
Your friend Bob will pick the last pile.
Repeat until there are no more piles of coins.
Given an array of integers piles where piles[i] is the number of coins in the ith pile.

Return the maximum number of coins which you can have.



```
Input: piles = [2,4,1,2,7,8]
Output: 9
Explanation: Choose the triplet (2, 7, 8), Alice Pick the pile with 8 coins, you the pile with 7 coins and Bob the last one.
Choose the triplet (1, 2, 4), Alice Pick the pile with 4 coins, you the pile with 2 coins and Bob the last one.
The maximum number of coins which you can have are: 7 + 2 = 9.
On the other hand if we choose this arrangement (1, 2, 8), (2, 4, 7) you only get 2 + 4 = 6 coins which is not optimal.
```



In [32]:
piles = [2,4,1,2,7,8]
alice = []
me = []
bob = []
def maxcoins(piles):
  return sum(sorted(piles)[len(piles)//3::2])

maxcoins(piles)


9